# Génération des Prédictions et Soumission - Challenge Titanic

Ce notebook final vous guide à travers les étapes pour générer des prédictions finales avec votre meilleur modèle et préparer votre soumission pour le challenge Titanic sur Kaggle.

## Objectifs:
- Charger le meilleur modèle sélectionné après l'évaluation
- Générer des prédictions sur l'ensemble de test
- Créer un fichier de soumission au format requis par Kaggle
- Soumettre les prédictions et analyser les résultats

In [ ]:
# Importation des bibliothèques nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
from sklearn.externals import joblib

# Configuration de l'affichage
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
sns.set(font_scale=1.2)

# Pour afficher les graphiques dans le notebook
%matplotlib inline

# Pour afficher toutes les colonnes
pd.set_option('display.max_columns', None)

## 1. Chargement des données prétraitées et du meilleur modèle

Commençons par charger les données de test prétraitées et le meilleur modèle identifié lors de l'étape d'évaluation.

In [ ]:
# Charger les données prétraitées
preprocessed_dir = '/workspaces/titanicML/Data/preprocessed'
preprocessed_file = os.path.join(preprocessed_dir, 'preprocessed_data.pkl')

try:
    # Charger les données prétraitées
    preprocessed_data = joblib.load(preprocessed_file)
    
    # Extraire les données de test et les IDs
    X_test = preprocessed_data['test_features']
    test_passenger_ids = preprocessed_data['test_passenger_ids']
    feature_names = preprocessed_data['feature_names']
    
    print("Données de test prétraitées chargées avec succès!")
    print(f"Ensemble de test: {X_test.shape[0]} observations, {X_test.shape[1]} caractéristiques")
    
except FileNotFoundError:
    print(f"Erreur: Fichier {preprocessed_file} introuvable.")
    print("Alternative: chargement des fichiers CSV...")
    
    try:
        test_file = os.path.join(preprocessed_dir, 'test_processed.csv')
        X_test = pd.read_csv(test_file)
        
        # Récupérer les IDs des passagers de test depuis le fichier d'origine
        test_data_original = pd.read_csv('/workspaces/titanicML/Data/test.csv')
        test_passenger_ids = test_data_original['PassengerId']
        
        feature_names = X_test.columns.tolist()
        
        print("Données de test chargées depuis les fichiers CSV.")
        print(f"Ensemble de test: {X_test.shape[0]} observations, {X_test.shape[1]} caractéristiques")
        
    except FileNotFoundError:
        print("Erreur: Fichiers CSV également introuvables.")
        print("Veuillez exécuter le notebook de prétraitement des données avant de continuer.")

# Rechercher les modèles disponibles
models_dir = '/workspaces/titanicML/models'
model_files = glob.glob(os.path.join(models_dir, 'final_model_*.pkl'))

if model_files:
    print(f"\nModèles disponibles: {len(model_files)}")
    
    # Charger chaque modèle
    models = {}
    for model_file in model_files:
        try:
            model_name = os.path.basename(model_file).replace('final_model_', '').replace('.pkl', '').replace('_', ' ')
            model_data = joblib.load(model_file)
            models[model_name] = model_data
            print(f"  - {model_name} chargé avec succès")
        except Exception as e:
            print(f"  - Erreur lors du chargement de {model_file}: {e}")
    
    # Vérifier si au moins un modèle a été chargé
    if models:
        print("\nModèles chargés avec succès!")
    else:
        print("\nAucun modèle n'a pu être chargé.")
else:
    print("\nAucun modèle trouvé dans le dossier des modèles.")
    print("Veuillez exécuter les notebooks de modélisation et d'évaluation avant de continuer.")

## 2. Sélection du meilleur modèle pour la soumission finale

Identifions le meilleur modèle à utiliser pour notre soumission finale, en nous basant sur les résultats de l'évaluation précédente.

In [ ]:
# Vérifier s'il existe un modèle d'ensemble qui pourrait être le meilleur choix
best_model = None
best_model_name = None

# Priorité au modèle d'ensemble s'il existe
if 'ensemble' in models:
    best_model = models['ensemble']['model']
    best_model_name = 'Ensemble'
    print("Le modèle d'ensemble a été sélectionné pour la soumission finale.")
    
# Si nous n'avons pas de modèle d'ensemble, charger les fichiers de soumission existants pour voir lequel a été le meilleur
else:
    submissions_dir = '/workspaces/titanicML/submissions'
    submission_files = glob.glob(os.path.join(submissions_dir, 'submission_*.csv'))
    
    if submission_files and models:
        # Si nous avons déjà des soumissions précédentes et des modèles chargés,
        # nous pourrions sélectionner le meilleur modèle en fonction d'une évaluation préalable
        # Pour cet exemple, nous allons simplement prendre le premier modèle disponible
        best_model_name = list(models.keys())[0]
        best_model = models[best_model_name]['model']
        print(f"Le modèle '{best_model_name}' a été sélectionné pour la soumission finale.")
    
    elif models:
        # Si nous n'avons pas de soumissions précédentes mais des modèles chargés
        best_model_name = list(models.keys())[0]
        best_model = models[best_model_name]['model']
        print(f"Le modèle '{best_model_name}' a été sélectionné pour la soumission finale (premier disponible).")
    
    else:
        # Si nous n'avons ni soumissions ni modèles, nous devons en créer un
        from sklearn.ensemble import RandomForestClassifier
        
        print("Aucun modèle n'a été trouvé. Création d'un modèle Random Forest par défaut...")
        
        # Charger les données d'entraînement
        try:
            X_train = preprocessed_data['train_features']
            y_train = preprocessed_data['train_target']
            
            # Créer et entraîner un modèle Random Forest simple
            best_model = RandomForestClassifier(n_estimators=100, random_state=42)
            best_model.fit(X_train, y_train)
            best_model_name = 'Random Forest (default)'
            
            print("Modèle Random Forest par défaut créé et entraîné avec succès.")
            
        except Exception as e:
            print(f"Erreur lors de la création du modèle par défaut: {e}")
            print("Veuillez exécuter les notebooks précédents pour créer des modèles.")
            # Nous ne pouvons pas continuer sans modèle
            raise

## 3. Génération des prédictions finales

Utilisons le meilleur modèle pour générer nos prédictions finales sur l'ensemble de test.

In [ ]:
# Vérifier que nous avons un modèle et des données de test
if best_model is not None and X_test is not None:
    # Générer les prédictions
    test_predictions = best_model.predict(X_test)
    
    # Si le modèle peut fournir des probabilités, les récupérer également
    if hasattr(best_model, 'predict_proba'):
        test_probabilities = best_model.predict_proba(X_test)[:, 1]
        has_probabilities = True
    else:
        test_probabilities = None
        has_probabilities = False
    
    # Créer le DataFrame pour la soumission
    submission = pd.DataFrame({
        'PassengerId': test_passenger_ids,
        'Survived': test_predictions.astype(int)
    })
    
    # Afficher les premières lignes de la soumission
    print("\nAperçu des prédictions finales:")
    display(submission.head(10))
    
    # Statistiques des prédictions
    survived_count = submission['Survived'].sum()
    total_count = len(submission)
    survival_rate = survived_count / total_count * 100
    
    print(f"\nStatistiques des prédictions:")
    print(f"Nombre de passagers prédits comme survivants: {survived_count} sur {total_count} ({survival_rate:.2f}%)")
    
    # Visualiser la distribution des prédictions
    plt.figure(figsize=(10, 6))
    sns.countplot(x='Survived', data=submission, palette=['#FF5252', '#4CAF50'])
    plt.title('Distribution des prédictions finales', fontsize=16)
    plt.xlabel('Prédit comme survivant (0 = Non, 1 = Oui)')
    plt.ylabel('Nombre de passagers')
    
    # Ajouter les annotations sur les barres
    counts = submission['Survived'].value_counts()
    for i, count in enumerate(counts):
        percentage = count / total_count * 100
        plt.text(i, count + 5, f'{count} ({percentage:.1f}%)', ha='center')
    
    plt.tight_layout()
    plt.show()
    
    # Si nous avons les probabilités, afficher leur distribution
    if has_probabilities:
        plt.figure(figsize=(10, 6))
        sns.histplot(data=test_probabilities, bins=30, kde=True)
        plt.axvline(x=0.5, color='r', linestyle='--', label='Seuil de décision')
        plt.title('Distribution des probabilités de survie', fontsize=16)
        plt.xlabel('Probabilité de survie')
        plt.ylabel('Nombre de passagers')
        plt.legend()
        plt.tight_layout()
        plt.show()
else:
    print("Impossible de générer des prédictions sans modèle ou données de test.")

## 4. Création et sauvegarde du fichier de soumission

Préparons le fichier de soumission au format requis par Kaggle et sauvegardons-le.

In [ ]:
# Créer le dossier pour les soumissions s'il n'existe pas
submissions_dir = '/workspaces/titanicML/submissions'
if not os.path.exists(submissions_dir):
    os.makedirs(submissions_dir)

# Créer un nom de fichier unique pour cette soumission
import datetime
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
model_name_safe = best_model_name.replace(' ', '_').lower() if best_model_name else 'unknown_model'
submission_file = os.path.join(submissions_dir, f'submission_{model_name_safe}_{timestamp}.csv')

# Sauvegarder le fichier de soumission
try:
    submission.to_csv(submission_file, index=False)
    print(f"Fichier de soumission créé avec succès: {submission_file}")
    
    # Créer également une copie avec un nom plus simple pour faciliter le téléchargement
    final_submission_file = os.path.join(submissions_dir, 'submission_final.csv')
    submission.to_csv(final_submission_file, index=False)
    print(f"Copie créée pour faciliter le téléchargement: {final_submission_file}")
    
    # Afficher les premières lignes du fichier pour vérification
    print("\nContenu du fichier de soumission:")
    display(pd.read_csv(submission_file).head())
    
    # Vérifier que le format est correct
    expected_columns = ['PassengerId', 'Survived']
    if all(col in submission.columns for col in expected_columns):
        print("\n✅ Le format du fichier de soumission est correct.")
        print("   Colonnes attendues: PassengerId, Survived")
        print(f"   Nombre de lignes: {len(submission)} (attendu: {len(test_passenger_ids)})")
        
        # Vérifier que les valeurs de Survived sont bien 0 ou 1
        valid_values = set(submission['Survived'].unique()).issubset({0, 1})
        if valid_values:
            print("   Valeurs de 'Survived' correctes (0 ou 1 uniquement)")
        else:
            print("❌ Erreur: Les valeurs de 'Survived' doivent être 0 ou 1 uniquement")
    else:
        missing_cols = [col for col in expected_columns if col not in submission.columns]
        print(f"\n❌ Erreur: Format du fichier incorrect. Colonnes manquantes: {missing_cols}")
except Exception as e:
    print(f"Erreur lors de la création du fichier de soumission: {e}")

## 5. Soumission à Kaggle

Pour soumettre vos prédictions à Kaggle, vous pouvez utiliser l'API Kaggle ou télécharger le fichier `submission_final.csv` et le soumettre manuellement sur le site web de Kaggle.

### 5.1 Soumission via l'API Kaggle

In [ ]:
# Vérifier que la configuration de l'API Kaggle est disponible
import os
from pathlib import Path

kaggle_json_path = os.path.join(Path.home(), '.kaggle', 'kaggle.json')

if os.path.exists(kaggle_json_path):
    print("Configuration de l'API Kaggle trouvée!")
    
    # Soumission à Kaggle en utilisant l'API
    print("\nSoumission des prédictions à Kaggle...")
    
    # La commande pour soumettre à Kaggle
    submission_command = f"kaggle competitions submit -c titanic -f {final_submission_file} -m \"Submission with {best_model_name} model\""
    
    # Exécuter la commande
    try:
        import subprocess
        result = subprocess.run(submission_command, shell=True, capture_output=True, text=True)
        
        if result.returncode == 0:
            print("\n✅ Soumission réussie!")
            print(result.stdout)
            
            # Vérifier le classement actuel
            print("\nVérification du classement actuel...")
            leaderboard_command = "kaggle competitions leaderboard -c titanic --show"
            leaderboard = subprocess.run(leaderboard_command, shell=True, capture_output=True, text=True)
            
            if leaderboard.returncode == 0:
                print("\nClassement actuel:")
                print(leaderboard.stdout)
            else:
                print("Impossible de récupérer le classement.")
                print(leaderboard.stderr)
        else:
            print("\n❌ Erreur lors de la soumission:")
            print(result.stderr)
            
            print("\nVous pouvez soumettre manuellement votre fichier en le téléchargeant et en l'important sur le site web de Kaggle.")
            print(f"Fichier à soumettre: {final_submission_file}")
    
    except Exception as e:
        print(f"\n❌ Erreur lors de l'exécution de la commande: {e}")
        print("\nVous pouvez soumettre manuellement votre fichier en le téléchargeant et en l'important sur le site web de Kaggle.")
        print(f"Fichier à soumettre: {final_submission_file}")
else:
    print("\n❌ Configuration de l'API Kaggle non trouvée.")
    print("Pour soumettre vos prédictions, vous pouvez:")
    print(f"1. Télécharger le fichier {final_submission_file}")
    print("2. Visiter https://www.kaggle.com/c/titanic/submit")
    print("3. Importer votre fichier de soumission")

## 6. Résumé du projet

Récapitulons tout le travail réalisé dans ce projet et les résultats obtenus.

In [ ]:
# Afficher un résumé du projet
print("=== RÉSUMÉ DU PROJET TITANIC ===\n")

# Informations sur les données
try:
    train_data = pd.read_csv('/workspaces/titanicML/Data/train.csv')
    test_data = pd.read_csv('/workspaces/titanicML/Data/test.csv')
    
    print(f"Ensemble d'entraînement: {train_data.shape[0]} observations, {train_data.shape[1]} colonnes")
    print(f"Ensemble de test: {test_data.shape[0]} observations, {test_data.shape[1]} colonnes")
    
    # Taux de survie dans les données d'entraînement
    survival_rate = train_data['Survived'].mean() * 100
    print(f"\nTaux de survie dans les données d'entraînement: {survival_rate:.2f}%")
    
except Exception as e:
    print(f"Impossible de charger les données d'origine: {e}")

# Informations sur le prétraitement
print("\nPrétraitement des données:")
print("- Imputation des valeurs manquantes pour l'âge, le port d'embarquement et le tarif")
print("- Création de nouvelles caractéristiques (taille de la famille, groupe d'âge, etc.)")
print("- Encodage des variables catégorielles")
print("- Normalisation des variables numériques")

# Informations sur les modèles
print("\nModèles testés:")
model_list = list(models.keys()) if 'models' in locals() and models else ["Aucun modèle trouvé"]
for model_name in model_list:
    print(f"- {model_name}")

# Informations sur la soumission finale
print("\nSoumission finale:")
if 'submission' in locals() and 'best_model_name' in locals() and best_model_name:
    print(f"- Modèle utilisé: {best_model_name}")
    print(f"- Taux de survie prédit: {submission['Survived'].mean()*100:.2f}%")
    print(f"- Fichier de soumission: {final_submission_file}")
else:
    print("- Aucune soumission générée")

print("\n=== FIN DU PROJET TITANIC ===")

## 7. Prochaines étapes et améliorations possibles

Voici quelques pistes pour améliorer davantage vos performances dans le challenge Titanic:

### Améliorations du prétraitement des données:
- Tester différentes stratégies d'imputation pour les valeurs manquantes
- Créer des caractéristiques plus complexes basées sur des interactions entre variables
- Explorer des techniques de sélection automatique des caractéristiques

### Améliorations des modèles:
- Tester d'autres algorithmes avancés comme les réseaux de neurones
- Optimiser davantage les hyperparamètres avec une recherche plus exhaustive
- Créer des ensembles plus sophistiqués combinant différents types de modèles
- Utiliser des techniques de validation croisée imbriquées pour une meilleure généralisation

### Améliorations de la soumission:
- Analyser les erreurs communes entre plusieurs modèles pour identifier des patterns
- Ajuster le seuil de décision pour optimiser le score sur Kaggle
- Tester plusieurs soumissions avec différents modèles et configurations

### Approfondissement de l'analyse:
- Explorer davantage les relations entre les variables et la survie
- Analyser les règles de décision apprises par les modèles
- Étudier la littérature historique sur le naufrage du Titanic pour incorporer des connaissances externes

## Conclusion

Félicitations! Vous avez maintenant complété l'ensemble du processus d'analyse et de prédiction pour le challenge Titanic sur Kaggle:

1. **Analyse exploratoire des données**: Vous avez exploré en détail les caractéristiques des passagers et leurs relations avec la survie
2. **Prétraitement des données**: Vous avez transformé les données brutes en caractéristiques exploitables par les modèles
3. **Modélisation**: Vous avez testé et optimisé différents algorithmes de machine learning
4. **Évaluation**: Vous avez analysé en profondeur les performances des modèles
5. **Soumission**: Vous avez généré des prédictions et soumis vos résultats à Kaggle

Ce projet vous a permis de mettre en pratique l'ensemble du cycle de vie d'un projet de science des données, de l'exploration initiale à la soumission finale. Les compétences acquises dans ce challenge peuvent être appliquées à de nombreux autres problèmes de classification en science des données.

N'hésitez pas à explorer les nombreuses ressources disponibles dans la communauté Kaggle pour continuer à améliorer vos performances sur ce challenge classique du machine learning!